In [9]:
import os
import tempfile

from common import *
import xml.etree.ElementTree as ET


In [3]:
def load_workflow(path: str) -> Graph:
    graph = get_graph()
    graph.parse(path, format='turtle')
    return graph

In [6]:
def get_workflow_steps(graph: Graph) -> list:
    steps = list(graph.subjects(RDF.type, dtbox.Step))
    return steps

In [ ]:
def get_base_config():
    root = ET.Element('config', {'xmlns': 'http://www.knime.org/2008/09/XMLConfig',
                                 'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
                                 'xsi:schemaLocation': 'http://www.knime.org/2008/09/XMLConfig http://www.knime.org/XMLConfig_2008_09.xsd',
                                 'key': 'settings.xml',
                                 })
    ET.SubElement(root, 'entry', {'key': 'node_file', 'type': 'xstring', 'value': 'settings.xml'})
    ET.SubElement(root, 'entry', {'key': 'customDescription', 'type': 'xstring', 'isnull': 'true', 'value': ''})
    #<entry key="state" type="xstring" value="CONFIGURED"/>
    ET.SubElement(root, 'entry', {'key': 'state', 'type': 'xstring', 'value': 'CONFIGURED'})
    #
    return root

In [ ]:
def create_step_file(ontology, workflow_graph, step, folder) -> None:
    pass

In [8]:
ontology = get_ontology_graph()
source_path = '../pipeline_generator/workflows/2023-07-15 21-40-38/workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24.ttl'
destination_path = './workflows/2023-07-15 20-27-10/workflow_10_DescriptionIntent_4762afff_2235_4c9e_b3ac_6876f5b634ed.knwf'

temp_folder = tempfile.mkdtemp()

In [7]:
graph = load_workflow(source_path)
steps = get_workflow_steps(graph)
print(steps)

[rdflib.term.URIRef('https://diviloper.dev/ontology/Workflow#workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24-step_0_component_random_absolute_train_test_split'), rdflib.term.URIRef('https://diviloper.dev/ontology/Workflow#workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24-step_1_component_decimal_scaling'), rdflib.term.URIRef('https://diviloper.dev/ontology/Workflow#workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24-step_3_component_drop_rows_with_missing_values'), rdflib.term.URIRef('https://diviloper.dev/ontology/Workflow#workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24-step_5_component_hypertangent_svm_learner'), rdflib.term.URIRef('https://diviloper.dev/ontology/Workflow#workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24-step_2_component_normalizer_applier'), rdflib.term.URIRef('https://diviloper.dev/ontology/Workflow#workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24-step_4_component_mis

In [ ]:
def create_step_file()

In [0]:
def translate_graph(ontology, source_path, destination_path):
    graph = load_workflow(source_path)
    components = get_workflow_components(graph)


In [4]:
def translate_graph_folder(ontology, source_folder, destination_folder):
    workflows = [f for f in os.listdir(source_folder) if f.endswith('.ttl')]
    for workflow in workflows:
        source_path = os.path.join(source_folder, workflow)
        destination_path = os.path.join(destination_folder, workflow[:-4] + '.knwf')
        translate_graph(ontology, source_path, destination_path)

<Graph identifier=N58bfe7a6d57f4156ab1905ae348c3d16 (<class 'rdflib.graph.Graph'>)>

In [ ]:
session = '2023-07-15 20-27-10'
translate_graph_folder(get_ontology_graph(), f'../pipline_generator/workflows/{session}', f'./workflows/{session}')